In [ ]:
! pip install langchain_community
! pip install langchain_text_splitter
! pip install langchain_openai
! pip install faiss-cpu

In [4]:
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()

import os

current_path = os.getcwd()
print("Current path:", current_path)

Current path: c:\Users\carpe\OneDrive\Desktop\p\rag


In [5]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

# CSV 파일 로드
loader = CSVLoader(file_path="recipies_data-241005.csv", encoding='utf-8')
data = loader.load()
print(f"Loaded {len(data)} documents from the csv.")

# 위에서 불러온 csv 데이터를 청크로 나눔
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(data)
len(splits)

# 벡터스토어 생성 및 로컬 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
vectorstore.save_local('./db/faiss')

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문 처리
question = "30분 이내에 만들 수 있는 중식 요리 방법 알려줘"
answer = rag_chain.invoke(question)
print(answer)

question = "토마토를 이용한 간단한 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

question = "고추를 이용한 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)


Loaded 5612 documents from the csv.
30분 이내에 만들 수 있는 중식 요리로는 '고메 중화짜장'이 있습니다. 아래는 요리 방법입니다.

1. 냄비에 물을 600ml 넣어 끓입니다.
2. 물이 끓으면 면을 넣고 2분 30초간 끓입니다.
3. 면을 건져낸 후 면수를 8숟가락 남겨둡니다.
4. 팬에 면을 넣고 비빔 소스를 넣어 볶습니다.
5. 수란, 오이, 청양고추 등을 곁들여 주면 완성입니다.

이 요리는 총 15분 정도 소요됩니다.
토마토를 이용한 간단한 요리로 "간단한 안주♥토마토카프레제"를 추천합니다. 이 요리는 4인분 기준으로, 준비 시간은 약 30분입니다. 필요한 재료는 다음과 같습니다:

- 토마토 2개
- 생 모짜렐라 치즈 2개
- 소금 약간
- 후추 약간
- 시판 발사믹 소스 1/2종이컵
- 다진 양파 1숟가락
- 어린잎 채소 약간

조리 방법은 다음과 같습니다:

1. 토마토와 생 모짜렐라 치즈를 슬라이스하여 준비합니다.
2. 토마토와 생 모짜렐라 치즈에 소금과 후추로 밑간을 합니다.
3. 볼에 소스 재료를 넣어 섞어 소스를 만듭니다.
4. 그릇에 토마토와 생 모짜렐라 치즈를 번갈아 가며 놓습니다.
5. 소스를 뿌리고 어린잎 채소를 올려 완성합니다.

더 자세한 레시피는 [여기](https://www.10000recipe.com//recipe/6890875)에서 확인할 수 있습니다.
고추를 이용한 요리로 "고추참치볶음밥"과 "고추닭볶음"을 추천합니다.

1. **고추참치볶음밥**
   - 재료: 밥 1인분, 고추참치 1캔, 옥수수콘 4숟가락, 달걀 1개, 소금 약간, 후추 약간, 식용유 약간, 청양고추 약간
   - 조리 방법:
     1. 팬에 기름을 두르고 달걀을 스크램블한다.
     2. 달걀이 반쯤 익으면 밥을 넣고 볶는다.
     3. 고추참치, 옥수수콘, 소금, 후추를 넣고 볶아 완성한다.
   - [레시피 링크](https://www.10000recipe.com//recipe/6918480)

2. *